In [1]:
# import data from VBOHCAR.xlsx to a pandas dataframe
import pandas as pd

# read the third sheet of the excel file
df = pd.read_excel('VBOHCAR.xlsx', sheet_name=3)

# print the first 5 rows
print(df.head())

   ID_OHCA        ReceivedTime  MinimumResponseTime   Latitude  Longitude  \
0        1 2017-01-01 00:49:54             5.666667  36.862471 -76.024169   
1        2 2017-01-01 06:35:29            20.750000  36.766897 -76.042337   
2        3 2017-01-01 06:56:25             9.650000  36.766897 -76.042337   
3        4 2017-01-01 17:05:56            11.083333  36.905880 -76.118769   
4        5 2017-01-01 19:53:20             0.000000  36.620850 -76.090090   

            Incident_Location       X_OHCA       Y_OHCA       Z_OHCA  
0     1000 LINDSLEY DR_x000D_  1232.449963 -4951.984772  3826.080166  
1  2600 SPRINGHAVEN DR_x000D_  1232.417401 -4958.562301  3817.562796  
2  2600 SPRINGHAVEN DR_x000D_  1232.417401 -4958.562301  3817.562796  
3        4200 SHORE DR_x000D_  1223.576354 -4951.197334  3829.945177  
4   5200 BLACKWATER RD_x000D_  1230.619605 -4969.017372  3804.526862  


In [2]:
# convert to geopandas dataframe
import geopandas as gpd
from shapely.geometry import Point

# create a geometry column from the coordinates
df['geometry'] = df.apply(lambda x: Point((float(x.Longitude), float(x.Latitude))), axis=1)

# create a geopandas dataframe
ohca_gdf = gpd.GeoDataFrame(df, geometry='geometry')

# print the first 5 rows
print(ohca_gdf.head())

   ID_OHCA        ReceivedTime  MinimumResponseTime   Latitude  Longitude  \
0        1 2017-01-01 00:49:54             5.666667  36.862471 -76.024169   
1        2 2017-01-01 06:35:29            20.750000  36.766897 -76.042337   
2        3 2017-01-01 06:56:25             9.650000  36.766897 -76.042337   
3        4 2017-01-01 17:05:56            11.083333  36.905880 -76.118769   
4        5 2017-01-01 19:53:20             0.000000  36.620850 -76.090090   

            Incident_Location       X_OHCA       Y_OHCA       Z_OHCA  \
0     1000 LINDSLEY DR_x000D_  1232.449963 -4951.984772  3826.080166   
1  2600 SPRINGHAVEN DR_x000D_  1232.417401 -4958.562301  3817.562796   
2  2600 SPRINGHAVEN DR_x000D_  1232.417401 -4958.562301  3817.562796   
3        4200 SHORE DR_x000D_  1223.576354 -4951.197334  3829.945177   
4   5200 BLACKWATER RD_x000D_  1230.619605 -4969.017372  3804.526862   

                     geometry  
0  POINT (-76.02417 36.86247)  
1  POINT (-76.04234 36.76690)  
2  POINT

In [3]:
# visualise on heatmap using folium
import folium

m = folium.Map([36.862471, -76.024169], zoom_start=11)

from folium.plugins import HeatMap

# convert to (n, 2) nd-array format for heatmap
coords = ohca_gdf[['Latitude', 'Longitude']].values

# plot heatmap
HeatMap(coords).add_to(m)
m

In [4]:
import overpass

# define the overpass api
api = overpass.API()

point_name = "church"

# define the query
query = """
area["name"="Virginia Beach"];
node["amenity"="place_of_worship"](area);
out;
"""
# get the response
response = api.get(query)

# convert to geopandas dataframe
churches = gpd.GeoDataFrame.from_features(response['features'])

# print the first 5 rows
print(churches.head())


                     geometry           amenity ele gnis:county_id  \
0  POINT (-76.04799 36.70960)  place_of_worship   2            810   
1  POINT (-76.08286 36.58601)  place_of_worship   3            810   
2  POINT (-76.07354 36.55765)  place_of_worship   2            810   
3  POINT (-76.16078 36.85042)  place_of_worship   5            810   
4  POINT (-76.02604 36.85598)  place_of_worship   7            810   

  gnis:created gnis:feature_id gnis:state_id                   name  \
0   09/28/1979         1478168            51     Beech Grove Church   
1   09/28/1979         1478227            51      Blackwater Church   
2   09/28/1979         1478340            51            Cava Church   
3   09/28/1979         1478583            51           Faith Church   
4   09/28/1979         1478593            51  First Colonial Church   

    religion denomination  ... addr:housenumber addr:street email phone  \
0  christian          NaN  ...              NaN         NaN   NaN   NaN   
1 

In [5]:
# use KDTree to find the nearest church to each OHCA
from scipy.spatial import KDTree

# convert to epsg:2283 coordinates
from pyproj import transformer

# define the transformer
transformer = transformer.Transformer.from_crs('epsg:4326', 'epsg:2283')

# convert to epsg:2283 coordinates
churches['X'] = churches.apply(lambda x: transformer.transform(x.geometry.y, x.geometry.x)[0], axis=1)
churches['Y'] = churches.apply(lambda x: transformer.transform(x.geometry.y, x.geometry.x)[1], axis=1)
ohca_gdf['X'] = ohca_gdf.apply(lambda x: transformer.transform(x.geometry.y, x.geometry.x)[0], axis=1)
ohca_gdf['Y'] = ohca_gdf.apply(lambda x: transformer.transform(x.geometry.y, x.geometry.x)[1], axis=1)

# create a KDTree from the church coordinates
ohca_tree = KDTree(ohca_gdf[['X', 'Y']].values)
church_tree = KDTree(churches[['X', 'Y']].values)

radius = 200

# query ball tree
churches['OHCA_ids'] = church_tree.query_ball_tree(ohca_tree, r=radius)

# convert list of indexes to a list of points
churches['OHCAs'] = churches.apply(lambda x: ohca_gdf.iloc[x.OHCA_ids].geometry.tolist(), axis=1)

# calculate how many ohcas happened within 200m of a point where the church was located
ohcas_near_church = sum(churches.OHCA_ids.apply(len))

# remove churches in the same location
churches = churches.drop_duplicates(subset='geometry')

print(f"Out of {len(ohca_gdf)} OHCAs {ohcas_near_church} have a {point_name} within {radius}m")
print(f"Out of {len(churches)} churches {len(churches[churches.OHCA_ids.apply(len) > 0])} have an OHCA within {radius}m")
print(f"That results in a mean ratio of {ohcas_near_church/len(churches)} OHCA per {point_name} across 3 years")
print(f"The {point_name} with the most OHCA cases had {max(churches.OHCA_ids.apply(len))} of them")
print(f"That's {max(churches.OHCA_ids.apply(len))/3/12:.2f} OHCA per month")
# show the church with the single most OHCA cases on the map
m = folium.Map([36.862471, -76.024169], zoom_start=11)

max_church = churches[churches.OHCA_ids.apply(len) == max(churches.OHCA_ids.apply(len))]
max_church = max_church.iloc[0]
# plot the church
folium.Marker([max_church.geometry.y, max_church.geometry.x], popup='Most OHCA cases').add_to(m)
m

Out of 2706 OHCAs 66 have a church within 200m
Out of 20 churches 4 have an OHCA within 200m
That results in a mean ratio of 3.3 OHCA per church across 3 years
The church with the most OHCA cases had 29 of them
That's 0.81 OHCA per month
